In [ ]:
# Libraries
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import math
import time
import random
import sklearn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import torch
from torch import nn
from torch import optim
from tqdm import tqdm
import torch.nn.functional as F

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Remove printing error
pd.options.mode.chained_assignment = None

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import pickle
from sklearn import metrics
from sklearn.metrics import roc_auc_score

from functions_folder.nn_MIMICDataset import MIMICDataset

import optuna
from optuna.trial import TrialState
import torch.optim as optim

In [2]:
# Set device
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

# Set the random seeds for deterministic results.
SEED = 0
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

GPU is available


In [3]:
# Data
mimic_data = pd.read_csv('mimic_data.csv')

# Split into 'preprocessing dataset' and a 'train/vaid/test dataset'
stays = mimic_data['stay_id'].unique()
import random
random.Random(0).shuffle(stays)
model_data2 = mimic_data.set_index("stay_id").loc[stays].reset_index()
n = round(0.5 * len(stays))
preprocessing_stays = stays[:n]
hold_out_stays = stays[n:]
preprocessing_data = model_data2[model_data2['stay_id'].isin(preprocessing_stays)]
hold_out_data = model_data2[model_data2['stay_id'].isin(hold_out_stays)]
preprocessing_data.drop(columns=['iv_treatment_length'], inplace=True)
preprocessing_data

,stay_id,date,iv_flag,first_po_flag,po_flag,blood pressure diastolic0,blood pressure diastolic1,blood pressure diastolic2,blood pressure diastolic3,blood pressure diastolic4,...,temperature14_current_stay_difference,temperature15_current_stay_difference,temperature16_current_stay_difference,temperature17_current_stay_difference,temperature18_current_stay_difference,temperature19_current_stay_difference,temperature20_current_stay_difference,temperature21_current_stay_difference,temperature_mean_current_stay_difference,temperature_std_current_stay_difference
0,39409066.0,2137-05-22,1.0,1.0,0,0.575521,0.519798,0.272727,0.700000,0.0,...,0.6,0.710059,0.600000,0.350276,-1.000000,-1.000000,0.467482,0.4,0.365151,0.075809
1,39409066.0,2137-05-23,1.0,1.0,0,0.448568,0.454810,0.454545,0.411765,0.0,...,0.4,0.410488,0.600000,0.551909,0.500693,0.439452,0.565014,0.4,0.482215,0.321606
2,30322661.0,2169-10-09,1.0,NaN,0,0.432292,0.446094,0.909091,0.304348,0.0,...,0.4,0.396435,0.600000,0.437195,0.526688,0.561250,0.666566,0.4,0.432688,0.363971
3,30322661.0,2169-10-10,1.0,1.0,0,0.361979,0.375081,0.545455,0.304348,0.0,...,0.4,0.396435,0.600000,0.437195,0.526688,0.561250,0.666566,0.4,0.432688,0.363971
4,30322661.0,2169-10-11,1.0,1.0,0,0.397135,0.378309,0.181818,0.500000,0.0,...,1.0,0.388391,0.642857,0.437195,0.526688,0.561250,0.358257,1.0,0.432688,0.664467
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14515,37575956.0,2153-02-15,1.0,1.0,0,0.479167,0.536475,0.090909,0.875000,0.0,...,0.6,0.568117,0.600000,0.479335,-1.000000,-1.000000,0.454375,0.4,0.295362,0.681000
14516,37575956.0,2153-02-16,1.0,1.0,0,0.522135,0.525500,0.272727,1.000000,0.0,...,0.6,0.568117,0.600000,0.479335,-1.000000,-1.000000,0.454375,0.4,0.295362,0.681000
14517,37540542.0,2161-05-10,1.0,NaN,0,0.253906,0.237573,0.090909,-1.000000,0.0,...,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.000000
14518,37540542.0,2161-05-11,1.0,1.0,0,0.397135,0.370562,0.363636,0.304348,0.0,...,0.6,0.562664,0.528571,-1.000000,-1.000000,-1.000000,-1.000000,0.4,0.234579,0.425554


In [4]:
all_shap_importance = pd.read_csv('all_shap_importance.csv')

shap_list = all_shap_importance[all_shap_importance['feature_importance_vals'] >= 0.5].col_name.unique().tolist()
shap_list = ['stay_id', 'date', 'iv_flag', 'first_po_flag', 'po_flag'] + shap_list
preprocessing_data = preprocessing_data[shap_list]
preprocessing_data

,stay_id,date,iv_flag,first_po_flag,po_flag,blood pressure systolic17_current_stay,heart rate17,respiratory rate11_current_stay,blood pressure mean11_current_stay,o2 saturation pulseoxymetry3_current_stay,...,blood pressure systolic12_current_stay,o2 saturation pulseoxymetry1_current_stay_difference,gcs - verbal response14_current_stay,temperature7_current_stay,temperature6_current_stay_difference,respiratory rate3,heart rate21_current_stay,o2 saturation pulseoxymetry7_difference,gcs - verbal response2_difference,heart rate8
0,39409066.0,2137-05-22,1.0,1.0,0,0.966557,-1.000000,0.761556,0.762332,1.000000,...,0.222222,0.542862,0.0,0.000000e+00,0.4,0.633333,0.000000,0.491376,-1.0,0.384615
1,39409066.0,2137-05-23,1.0,1.0,0,0.431357,0.343572,0.126674,0.539524,0.573477,...,0.333333,0.599999,0.0,1.083738e-28,0.4,0.745098,0.111111,0.279022,-1.0,0.615385
2,30322661.0,2169-10-09,1.0,NaN,0,0.414066,0.230392,0.095854,0.400227,0.583127,...,0.111111,0.599999,0.0,9.099612e-29,0.4,0.527778,0.148148,0.364522,-1.0,0.076923
3,30322661.0,2169-10-10,1.0,1.0,0,0.327966,0.221895,0.046076,0.280141,0.460829,...,0.333333,0.599999,0.0,0.000000e+00,0.4,0.275362,0.259259,0.364522,-1.0,0.153846
4,30322661.0,2169-10-11,1.0,1.0,0,0.248910,0.521848,0.032439,0.198931,0.430108,...,0.333333,0.599999,0.0,1.622159e-01,0.4,0.452381,0.592593,0.571676,-1.0,0.230769
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14515,37575956.0,2153-02-15,1.0,1.0,0,0.988171,-1.000000,0.953186,0.569251,-1.000000,...,0.000000,0.142857,-1.0,-1.000000e+00,0.4,0.791667,0.000000,0.439876,0.0,0.230769
14516,37575956.0,2153-02-16,1.0,1.0,0,0.626321,-1.000000,0.166640,0.800980,0.774194,...,0.111111,0.142857,-1.0,-1.000000e+00,0.4,0.904762,0.111111,0.439876,0.0,0.538462
14517,37540542.0,2161-05-10,1.0,NaN,0,0.968004,-1.000000,0.761556,0.718315,-1.000000,...,0.111111,-1.000000,-1.0,-1.000000e+00,-1.0,-1.000000,0.000000,-1.000000,-1.0,-1.000000
14518,37540542.0,2161-05-11,1.0,1.0,0,0.606376,0.521273,0.387155,0.412453,0.564516,...,0.333333,0.085715,0.0,2.895331e-29,-1.0,0.263889,0.259259,-1.000000,-1.0,0.307692


In [5]:
individual = '110111100011000100010101000000001011100101111010000000001100010100101000011100000010101110001'

In [6]:
stays = preprocessing_data['stay_id'].unique()
import random
random.Random(5).shuffle(stays) # Note changed random split to 5 so split between po_flag is similar for val and test sets 
X_data = preprocessing_data.drop(columns=['stay_id', 'po_flag', 'date', 'iv_flag', 'first_po_flag'])
# Filter for features in this individual
X_data = X_data.loc[:, [True if individual[i] == '1' else False for i in range(len(individual))]]
model_data = pd.concat([preprocessing_data[['stay_id', 'po_flag']], X_data], axis=1)
model_data2 = model_data.set_index("stay_id").loc[stays].reset_index()
n = round(0.7 * len(stays))
n2 = round(0.85 * len(stays))
train_stays = stays[:n]
validation_stays = stays[n:n2]
test_stays = stays[n2:]
train_data = model_data2[model_data2['stay_id'].isin(train_stays)]
valid_data = model_data2[model_data2['stay_id'].isin(validation_stays)]
test_data = model_data2[model_data2['stay_id'].isin(test_stays)]

In [7]:
preprocessing_data.po_flag.value_counts(normalize=True)
hold_out_data.po_flag.value_counts(normalize=True)
len(train_data)
train_data.po_flag.value_counts(normalize=True)
len(valid_data)
valid_data.po_flag.value_counts(normalize=True)
len(test_data)
test_data.po_flag.value_counts(normalize=True)

0    0.836019
1    0.163981
Name: po_flag, dtype: float64

0    0.83458
1    0.16542
Name: po_flag, dtype: float64

10207

0    0.840404
1    0.159596
Name: po_flag, dtype: float64

2127

0    0.827927
1    0.172073
Name: po_flag, dtype: float64

2186

0    0.823422
1    0.176578
Name: po_flag, dtype: float64

Optimize

In [8]:
# Oversampling train set
from typing import Counter
import imblearn
from imblearn.over_sampling import SMOTE
# Split X y
train_data_X = train_data.drop(columns=['stay_id', 'po_flag'])
train_data_y = train_data['po_flag']
Counter(train_data_y)
oversample = SMOTE()
train_data_X, train_data_y = oversample.fit_resample(train_data_X, train_data_y)
Counter(train_data_y)
train_data_y = pd.DataFrame(train_data_y, columns=['po_flag'])
train_data_y['stay_id'] = 'x'
train_data = pd.concat([train_data_y, train_data_X], axis=1)
train_data = train_data.sample(frac=1, random_state=0).reset_index(drop=True)

Counter({0: 8578, 1: 1629})

Counter({0: 8578, 1: 8578})

In [14]:
def define_model(trial):
    # We optimize the number of layers, hidden units and dropout ratio in each layer.
    n_layers = trial.suggest_int("n_layers", 1, 3)
    layers = []
    in_features = len(train_data.columns)-2
    layers.append(nn.BatchNorm1d(in_features))
    p = trial.suggest_categorical("dropout", [0, 0.1, 0.2, 0.3, 0.4, 0.5])

    for i in range(n_layers):
        out_features = trial.suggest_categorical("n_units_l{}".format(i), [2, 4, 8, 16, 32, 64, 128, 256, 512])
        #layers.append(nn.BatchNorm1d(in_features))
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.ReLU())
        #p = trial.suggest_float("dropout_l{}".format(i), 0, 0.5)
        layers.append(nn.Dropout(p))

        in_features = out_features
    layers.append(nn.Linear(in_features, 1))

    return nn.Sequential(*layers)

In [15]:
def objective(trial):

    # Generate the model.
    model = define_model(trial).to(device)

    # Generate the optimizers.
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam"])
    #lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    lr = trial.suggest_categorical("lr", [0.001, 0.0001])
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)
    batch_size = trial.suggest_categorical("batch_size", [256, 512])

    # Get the dataset.
    train_dataset = MIMICDataset(train_data)
    train_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size, collate_fn=train_dataset.collate_fn_padd)

    valid_dataset = MIMICDataset(valid_data)
    valid_dataloader = DataLoader(dataset=valid_dataset, batch_size=batch_size, collate_fn=valid_dataset.collate_fn_padd)

    #test_dataset = MIMICDataset(test_data)
    #test_dataloader = DataLoader(dataset=test_dataset, batch_size=batch_size, collate_fn=test_dataset.collate_fn_padd)

    # Define loss
    criterion = nn.BCEWithLogitsLoss()

    best_valid_auroc = 0

    # Training of the model.
    for epoch in range(EPOCHS):
        model.train()
        epoch_loss = 0
        batch_prediction_list = []
        batch_label_list = []
        for batch_idx, (labels, features) in enumerate(train_dataloader):
            features = features.float()
            labels = labels.float()
            features = features.to(device=device)
            labels = labels.to(device=device)
            optimizer.zero_grad()
            features2 = torch.permute(features, (1, 0))
            labels2 = torch.permute(labels, (1, 0))
            output = model(features2)
            loss = criterion(output, labels2)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
            sig = torch.nn.Sigmoid()
            output = sig(output)      
            np_predictions = output.cpu().detach().numpy()
            np_labels = labels2.cpu().detach().numpy()
            np_predictions = np_predictions.squeeze()
            np_labels = np_labels.squeeze()
            np_predictions = np_predictions.flatten()
            np_labels = np_labels.flatten()
            for x in np_predictions:
                batch_prediction_list.append(x)
            for x in np_labels:
                batch_label_list.append(x)
        final_predictions = np.array(batch_prediction_list)
        final_labels = np.array(batch_label_list)
        train_auroc = roc_auc_score(final_labels, final_predictions.round())
        train_loss = epoch_loss / len(train_dataloader)
        print('epoch:', epoch)
        print('train_auroc:', train_auroc)
        print('train_loss:', train_loss)

        # Validation of the model.
        model.eval()
        epoch_loss = 0
        batch_prediction_list = []
        batch_label_list = []
        with torch.no_grad():
            for batch_idx, (labels, features) in enumerate(valid_dataloader):
                features = features.float()
                labels = labels.float()
                features = features.to(device=device)
                labels = labels.to(device=device)
                features2 = torch.permute(features, (1, 0))
                labels2 = torch.permute(labels, (1, 0))
                output = model(features2)
                loss = criterion(output, labels2)
                epoch_loss += loss.item()
                sig = torch.nn.Sigmoid()
                output = sig(output)  
                np_predictions = output.cpu().detach().numpy()
                np_labels = labels2.cpu().detach().numpy()
                np_predictions = np_predictions.squeeze()
                np_labels = np_labels.squeeze()
                np_predictions = np_predictions.flatten()
                np_labels = np_labels.flatten()
                # Create list
                for x in np_predictions:
                    batch_prediction_list.append(x)
                for x in np_labels:
                    batch_label_list.append(x)
        final_predictions = np.array(batch_prediction_list)
        final_labels = np.array(batch_label_list)
        valid_auroc = roc_auc_score(final_labels, final_predictions.round())
        valid_loss = epoch_loss / len(train_dataloader)
        print('valid_auroc:', valid_auroc)
        print('valid_loss:', valid_loss)
        #print(type(valid_auroc))

        trial.report(valid_auroc, epoch)

        if valid_auroc > best_valid_auroc:
            best_valid_auroc = valid_auroc

        if valid_auroc == 0.5: # Could change to less than 0.6 or something to be more stringent
            raise optuna.exceptions.TrialPruned()

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return best_valid_auroc

In [16]:
EPOCHS = 10
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics:")
print("Number of finished trials:", len(study.trials))
print("Number of pruned trials:", len(pruned_trials))
print("Number of complete trials:", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("Value:", trial.value)

print("Params:")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2022-10-13 17:45:54,744] A new study created in memory with name: no-name-cc54d095-c14b-4667-ae56-bf113db07488


epoch: 0
train_auroc: 0.49784332012124033
train_loss: 0.6745895936208612
valid_auroc: 0.49630891538898353
valid_loss: 0.08289675502216115
epoch: 1
train_auroc: 0.5047213802751224
train_loss: 0.6621986294493956
valid_auroc: 0.5044521089917241
valid_loss: 0.08049321174621582
epoch: 2
train_auroc: 0.5156213569596643
train_loss: 0.6502175418769612
valid_auroc: 0.5155044171996165
valid_loss: 0.07908761150696698
epoch: 3
train_auroc: 0.5338074143156913
train_loss: 0.6386248013552498
valid_auroc: 0.5455753530501485
valid_loss: 0.07781843928729787
epoch: 4
train_auroc: 0.5577640475635346
train_loss: 0.6272524665383732
valid_auroc: 0.5665442511240819
valid_loss: 0.07666656024315778
epoch: 5
train_auroc: 0.5978666355793891
train_loss: 0.616029225728091
valid_auroc: 0.6155802869085188
valid_loss: 0.07563825565225937
epoch: 6
train_auroc: 0.6360456982979715
train_loss: 0.6035171501776752
valid_auroc: 0.6538107074035804
valid_loss: 0.07472836445359622
epoch: 7
train_auroc: 0.6681044532525063
train_

[I 2022-10-13 17:48:45,213] Trial 0 finished with value: 0.7272879604546597 and parameters: {'n_layers': 1, 'dropout': 0.2, 'n_units_l0': 16, 'optimizer': 'Adam', 'lr': 0.0001, 'batch_size': 512}. Best is trial 0 with value: 0.7272879604546597.


valid_auroc: 0.7272879604546597
valid_loss: 0.07268443177728091
epoch: 0
train_auroc: 0.7846817439962694
train_loss: 0.45160504583926764
valid_auroc: 0.8167234525837593
valid_loss: 0.04975318645729738
epoch: 1
train_auroc: 0.8210538587083236
train_loss: 0.38162290304899216
valid_auroc: 0.8179662263430799
valid_loss: 0.04803850852391299
epoch: 2
train_auroc: 0.8331778969456751
train_loss: 0.3624260717016809
valid_auroc: 0.821035924074436
valid_loss: 0.04724329008775599
epoch: 3
train_auroc: 0.8412217300069946
train_loss: 0.349412820466301
valid_auroc: 0.8189785982256729
valid_loss: 0.04684382764732137
epoch: 4
train_auroc: 0.8476917696432735
train_loss: 0.3381371214054525
valid_auroc: 0.8162998854972492
valid_loss: 0.046464683378444
epoch: 5
train_auroc: 0.8532291909536022
train_loss: 0.3288870438495103
valid_auroc: 0.8161230113292559
valid_loss: 0.0459758477175937
epoch: 6
train_auroc: 0.8596992305898812
train_loss: 0.31883670289672034
valid_auroc: 0.810888932331667
valid_loss: 0.04590

[I 2022-10-13 17:51:30,257] Trial 1 finished with value: 0.821035924074436 and parameters: {'n_layers': 1, 'dropout': 0.2, 'n_units_l0': 512, 'optimizer': 'Adam', 'lr': 0.001, 'batch_size': 256}. Best is trial 1 with value: 0.821035924074436.


...

valid_auroc: 0.8163650496644046
valid_loss: 0.05913687190588783
epoch: 0
train_auroc: 0.5735602704593146
train_loss: 0.6683785591055366


[I 2022-10-13 18:57:05,345] Trial 48 pruned. 


valid_auroc: 0.7673895234637548
valid_loss: 0.07122948415139142
epoch: 0
train_auroc: 0.6258451853578924
train_loss: 0.6471455666948768


[I 2022-10-13 18:57:21,033] Trial 49 pruned. 


valid_auroc: 0.7682529486785639
valid_loss: 0.07097672890214358
Study statistics:
Number of finished trials: 50
Number of pruned trials: 27
Number of complete trials: 23
Best trial:
Value: 0.8323512162426341
Params:
    n_layers: 3
    dropout: 0.5
    n_units_l0: 64
    n_units_l1: 512
    n_units_l2: 256
    optimizer: Adam
    lr: 0.001
    batch_size: 256
